In [ ]:
import ee
from IPython.display import Image
import geemap
from google.colab import files

# Autenticar
ee.Authenticate()
ee.Initialize(project='invertible-env-459922-h2')

# 1. Definir punto de interés (Rosario)
coordenadas = [-60.91028118699678, -32.92654351800582]  # [longitud, latitud]
punto = ee.Geometry.Point(coordenadas)

# 2. Obtener imagen Sentinel-2
imagen_26mar = (ee.ImageCollection('COPERNICUS/S2_SR')
          .filterBounds(punto)
          .filterDate('2025-03-26', '2025-03-27')
          .sort('CLOUDY_PIXEL_PERCENTAGE')
          .first())

# 3. Calcular NDVI = (NIR - Red) / (NIR + Red)
# Bandas Sentinel-2: B8 = NIR (infrarrojo cercano), B4 = Red (rojo)
ndvi_26mar = imagen_26mar.normalizedDifference(['B8', 'B4']).rename('NDVI')

# 4. Parámetros de visualización del NDVI
ndvi_params = {
    'min': -1,   # Valores posibles de NDVI
    'max': 1,    # -1 a 1 (vegetación sana ~0.6-0.9)
    'palette': ['red', 'yellow', 'green']  # Escala de colores
}

# 5. Generar y mostrar imagen NDVI (área de 2km alrededor)
Image(url=ndvi_26mar.getThumbUrl({
    'region': punto.buffer(17000).bounds(),
    'dimensions': 800,
    'format': 'png',
    **ndvi_params
}))

# 1. Definir punto de interés (Rosario)
coordenadas = [-60.836417103516666, -32.92654351800582]  # [longitud, latitud]
punto = ee.Geometry.Point(coordenadas)

# 2. Obtener imagen Sentinel-2
imagen_05abr = (ee.ImageCollection('COPERNICUS/S2_SR')
          .filterBounds(punto)
          .filterDate('2025-04-05', '2025-04-06')
          .sort('CLOUDY_PIXEL_PERCENTAGE')
          .first())

# 3. Calcular NDVI = (NIR - Red) / (NIR + Red)
# Bandas Sentinel-2: B8 = NIR (infrarrojo cercano), B4 = Red (rojo)
ndvi_05abr = imagen_05abr.normalizedDifference(['B8', 'B4']).rename('NDVI')

# 4. Parámetros de visualización del NDVI
ndvi_params = {
    'min': -1,   # Valores posibles de NDVI
    'max': 1,    # -1 a 1 (vegetación sana ~0.6-0.9)
    'palette': ['red', 'yellow', 'green']  # Escala de colores
}

# 5. Generar y mostrar imagen NDVI (área de 2km alrededor)
Image(url=ndvi_05abr.getThumbUrl({
    'region': punto.buffer(17000).bounds(),
    'dimensions': 800,
    'format': 'png',
    **ndvi_params
}))


# Definir una región común como un buffer circular de 17 km alrededor del punto
region_comun = punto.buffer(17000).bounds()

# 1. Cálculo del cambio relativo con suavizado
def calcular_cambio_relativo_suavizado(ndvi_inicial, ndvi_final, radio_blur=150):
    """
    Calcula el cambio relativo con suavizado espacial
    Args:
        radio_blur: radio del suavizado en metros (mayor = más suave)
    """
    # Calcular diferencia absoluta
    diferencia = ndvi_final.subtract(ndvi_inicial)

    # Calcular cambio relativo (%)
    cambio_rel = diferencia.divide(ndvi_inicial.add(0.001)).multiply(100)

    # Aplicar suavizado con filtro de media
    cambio_suavizado = cambio_rel.focal_mean(
        radius=radio_blur,
        units='meters',
        kernelType='circle'
    ).rename('cambio_rel_suavizado')

    # Máscara para áreas con vegetación inicial significativa
    mascara_veg = ndvi_inicial.gt(0)

    return cambio_suavizado.updateMask(mascara_veg)

# 2. Aplicar función a tus imágenes
cambio_suavizado = calcular_cambio_relativo_suavizado(ndvi_26mar, ndvi_05abr, 20)  # Radio de 200m

# 3. Parámetros de visualización mejorados
paleta_colores = [
    '#d73027',  # rojo oscuro (pérdida severa)
    '#f46d43',  # rojo claro
    '#fdae61',  # naranja
    '#fee08b',  # amarillo
    '#ffffbf',  # neutro
    '#d9ef8b',  # verde claro
    '#a6d96a',  # verde
    '#66bd63',  # verde medio
    '#1a9850'   # verde oscuro (ganancia severa)
]

vis_params = {
    'min': -40,  # -60% de cambio
    'max': 40,   # +60% de cambio
    'palette': paleta_colores,
    'opacity': 0.8
}

# 4. Visualización interactiva
Map = geemap.Map(center=[-32.9265, -60.8733], zoom=12)

# Capa base: NDVI inicial
Map.addLayer(ndvi_26mar.clip(region_comun),
           {'min': 0, 'max': 1, 'palette': ['white', 'green']},
           'NDVI Inicial (Base)')

# Capa adicional: NDVI después del evento
Map.addLayer(ndvi_05abr.clip(region_comun),
           {'min': 0, 'max': 1, 'palette': ['white', 'green']},
           'NDVI Después del Evento')

# Para imágenes multibanda (ej: Sentinel-2, Landsat)
Map.addLayer(imagen_26mar.clip(region_comun).select(['B4', 'B3', 'B2']),
           {'min': 0, 'max': 3000},  # Ajusta los valores según tu imagen
           'Imagen antes del Evento (RGB)')

Map.addLayer(imagen_05abr.clip(region_comun).select(['B4', 'B3', 'B2']),
           {'min': 0, 'max': 3000},
           'Imagen después del Evento (RGB)')


# Capa principal: Cambio relativo suavizado
Map.addLayer(cambio_suavizado.clip(region_comun), vis_params, 'Cambio Relativo Suavizado (200m)')

# Control de capas y leyenda

Map.add_colorbar(
    vis_params,
    label="Cambio Relativo en Vegetación (%)",
    layer_name="Cambio Relativo Suavizado (200m)",
    orientation="horizontal",
    transparent_bg=True
)

# 5. Análisis de áreas con mayores cambios
print("Áreas con cambios más significativos:")
stats_extremos = cambio_suavizado.reduceRegion(
    ee.Reducer.percentile([5, 25, 50, 75, 95]),
    region_comun,
    100
).getInfo()

print(f"• Pérdida máxima: {stats_extremos['cambio_rel_suavizado_p5']:.1f}%")
print(f"• Pérdida típica: {stats_extremos['cambio_rel_suavizado_p25']:.1f}%")
print(f"• Cambio mediano: {stats_extremos['cambio_rel_suavizado_p50']:.1f}%")
print(f"• Ganancia típica: {stats_extremos['cambio_rel_suavizado_p75']:.1f}%")
print(f"• Ganancia máxima: {stats_extremos['cambio_rel_suavizado_p95']:.1f}%")

Map


# Guardar como archivo HTML interactivo
Map.to_html('mapa_ndvi_interactivo.html', title='Análisis de Cambio Relativo NDVI')

from google.colab import files
files.download('mapa_ndvi_interactivo.html')

